In [1]:
from hello import hello, hello3
from officelib.xllib import *
from officelib import const

In [20]:
HERR = (hello.HelloError, hello3.HelloError)

from time import sleep
def set_mg(sp):
    try:
        app.setmg(1, sp)
    except HERR:
        app.login()
        app.setmg(1, sp)
        
def update_ss(cell, val):
    cell.Value = val
    c = cell.Offset(2, 1)
    c.Select()
    return c
        
def frange(start, stop, step):
    i = 0
    v = start + i * step
    while v < stop:
        yield v
        i += 1
        v = start + i * step
        

def run_test(name, mfc_max, c=None):
    tmin = mfc_max / 10
    print("Testing ", name, "MFC: %.1f LPM Max" % mfc_max)
    app.login()
    for fr in frange(tmin, mfc_max+tmin/2, tmin):
        set_mg(fr)
        v = input("%.2f LPM:" % fr)
        if v.lower() in ("break", "quit", "q", "stop"):
            break
        c = update_ss(c, v)

def air_on():
    app.setph(2,0,0)
    app.setdo(2,0,0)
    
def n2_on():
    app.setph(2,0,0)
    app.setdo(1, 100, 0)
    
def o2_on():
    app.setph(2,0,0)
    app.setdo(1, 0, 100)
    
def co2_on():
    app.setconfig("pH", "CO2_Manual_Max_(%)", 100)
    app.setph(1, 100, 0)
    app.setdo(2, 0, 0)
    app.setmg(1, 0.1)
    input("Purging line of air. Press enter when done.")
    

cc = {
    "co2": 4,
    "air": 1,
    "n2": 2,
    "o2": 3
}

on_fn = {
    "co2": co2_on,
    "air": air_on,
    "n2": n2_on,
    "o2": o2_on
}

    
def s():
    return xl.Selection

def run(name, mfcm):
    c = cc[name]
    app.login()
    if name == "co2":
        co2m = app.getConfig()['pH']['CO2_Manual_Max_(%)']
    on_fn[name]()
    run_test(name, mfcm, cells(2,c))
    if name == "co2":
        app.login()
        app.setconfig("pH", "CO2_Manual_Max_(%)", co2m)

def all_off():
    app.login()
    app.setag(0, 12)
    app.setpumpa(0, 0)
    app.setpumpb(0, 0)
    app.setpumpc(0, 0)
    app.setpumpsample(0, 0)
    
def connect():
    global app
    v = input("Version: ").lower()
    ip = input("Enter IP Address: ")
    if v == "3":
        app = hello3.HelloApp(ip)
    else:
        app = hello.HelloApp(ip)
    
def init():
    global xl, wb, ws, cells, cell_range
    xl, wb, ws, cells = xlObjs()
    cell_range = cells.Range
    cell_range("A1:D1").Value = [("Air", "N2", "O2", "CO2")]
    

In [3]:
init()
connect()
all_off()

Version: 3
Enter IP Address: 192.168.1.4


In [6]:
run("air", .5)

Testing  air MFC: 0.5 LPM Max
0.05 LPM:.0561
0.10 LPM:.1072
0.15 LPM:.1584
0.20 LPM:.2098
0.25 LPM:.2616
0.30 LPM:.3145
0.35 LPM:.3658
0.40 LPM:.4163
0.45 LPM:.4664
0.50 LPM:.5143


In [7]:
run("n2", 0.5)

Testing  n2 MFC: 0.5 LPM Max
0.05 LPM:.0564
0.10 LPM:.1078
0.15 LPM:.159
0.20 LPM:.2108
0.25 LPM:.2628
0.30 LPM:.3158
0.35 LPM:.3678
0.40 LPM:.4192
0.45 LPM:.4693
0.50 LPM:.5175


In [5]:
run("o2", .5)

Testing  o2 MFC: 0.5 LPM Max
0.05 LPM:.0573
0.10 LPM:.1079
0.15 LPM:.1596
0.20 LPM:.2102
0.25 LPM:.2628
0.30 LPM:.3163
0.35 LPM:.3668
0.40 LPM:.418
0.45 LPM:.469
0.50 LPM:.5172


In [21]:
run("co2", .1)

Purging line of air. Press enter when done.
Testing  co2 MFC: 0.1 LPM Max
0.01 LPM:0
0.02 LPM:0
0.03 LPM:0.0305
0.04 LPM:0.041.2
0.05 LPM:0.0521
0.06 LPM:0.0625
0.07 LPM:0.0731
0.08 LPM:0.0836
0.09 LPM:0.0943
0.10 LPM:0.103
